In [60]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.transforms import Affine2D
import time
from matplotlib.offsetbox import AnchoredText 
import mplcursors 
import os
import json
from gearbox import Gearbox

import itertools
import math

from tqdm import tqdm

In [61]:
import sys

start_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils import score_vals, param_to_list

In [62]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4" # select number of desired GPU to be used
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

## To Move into .py helper file:

variables: z_s, z_p2, z_r2, xs, xp1, xp2, xr2

In [63]:
config = {
  "MODULE": 0.8e-3,
  "MC": 0.8467e-3,
  "MU": 0.21,
  "ALPHA": 20,
  "XS": 0.476,
  "XP1": 0.762,
  "XP2": 0.536,
  "XR1": 2,
  "XR2": 1.21,
  "XS_RANGE": [-0.5, 0.5, 0.25],
  "XP1_RANGE": [-1, 1, 0.25],
  "XP2_RANGE": [-0.5, 0.5, 0.25],
  "XR2_RANGE": [-1.5, 1.5, 0.25],
  "N_PLANETS": 3,
  "SUN_LIMITS": [4, 26],
  "P1_LIMITS": [32, 42],
  "P2_LIMITS": [25, 35],
  "R2_LIMITS": [70,86],
  "STEPS": [2, 1, 1, 1],
  "MIN_RATIO_THRESHOLD": 90,
  "MIN_FORWARD_THRESHOLD": 0.30,
  "MIN_BACKWARD_THRESHOLD": 0.30
}


In [64]:
MODULE = config["MODULE"]
MU = config["MU"]
MC = config.get("MC", MODULE)
ALPHA = np.radians(config.get("ALPHA", 20))
XR1 = config.get("XR1", 0.0)
N_PLANETS = config["N_PLANETS"]
SUN_LIMITS = tuple(config["SUN_LIMITS"])
P1_LIMITS  = tuple(config["P1_LIMITS"])
P2_LIMITS  = tuple(config["P2_LIMITS"])
R2_LIMITS  = tuple(config["R2_LIMITS"])
STEPS = tuple(config["STEPS"])
MIN_RATIO_THRESHOLD = config["MIN_RATIO_THRESHOLD"]
MIN_FORWARD_THRESHOLD = config["MIN_FORWARD_THRESHOLD"]
MIN_BACKWARD_THRESHOLD = config["MIN_BACKWARD_THRESHOLD"]

SLICE_GEAR_RATIO = True 
TARGET_GEAR_RATIO = 100
RATIO_TOLERANCE = 10
DISPLAY_BEST_GEARBOX = False
DISPLAY_ITERATIONS_PLOT = True
RUN_SAMPLE_TRIAL = False
MAX_CANDIDATES = 1000

In [65]:
def generate_options(lower: int, upper: int, step: int = 1):
    return list(range(lower, upper + 1, step))

def generate_range_options(range_vals):
    low, high, step = range_vals
    opts = []
    current = low
    while current <= high + 1e-9:
        opts.append(round(current, 5))
        current += step
    return opts

def generate_all_options(sun_limits: tuple, p1_limits: tuple, p2_limits: tuple, r2_limits: tuple, 
                         steps: tuple = (1,1,1,1)):
    step_sun, step_p1, step_p2, step_r2 = steps
    return {
        "sun": generate_options(sun_limits[0], sun_limits[1], step_sun),
        "p1": generate_options(p1_limits[0], p1_limits[1], step_p1),
        "p2": generate_options(p2_limits[0], p2_limits[1], step_p2),
        "r2": generate_options(r2_limits[0], r2_limits[1], step_r2)
    }

In [66]:
options = generate_all_options(SUN_LIMITS, P1_LIMITS, P2_LIMITS, R2_LIMITS, STEPS)
sun_options = options["sun"]
p1_options  = options["p1"]
p2_options  = options["p2"]
r2_options  = options["r2"]
xs_options = generate_range_options(config["XS_RANGE"])
xp1_options = generate_range_options(config["XP1_RANGE"])
xp2_options = generate_range_options(config["XP2_RANGE"])
xr2_options = generate_range_options(config["XR2_RANGE"])

In [67]:
# Working vals from BayZop (3 with gear from lit):
BayzOp_Highest = {
'z_sh': 15, 'z_sh_1': 11, 'z_sh_2': 10, 'z_r2': 72, 'z_r2_1': 71, 'z_r2_2': 70, 'x_s': 1.0, 'x_s_1': 0.380826, 'x_s_2': 0.280826, 'x_r2': -0.423252, 'x_r2_1': -0.523252, 'x_r2_2': -1.0
}
BayzOp_Optimal = {
    'z_sh': 15,
    'z_sh_1': 10,
    'z_sh_2': 9,
    'z_r2': 72,
    'z_r2_1': 71,
    'z_r2_2': 70,
    'x_s': 0.9999999999960997,
    'x_s_1': 0.8999999999149966,
    'x_s_2': 0.7999999998281693,
    'x_r2': -0.25155638920287665,
    'x_r2_1': -0.351556389257909,
    'x_r2_2': -0.9999999999177834,
}

BayzOp_4_Optimal = {
    'z_sh': 9,
    'z_sh_1': 8,
    'z_sh_2': 7,
    'z_sh_3': 6,
    'z_r2': 75,
    'z_r2_1': 74,
    'z_r2_2': 73,
    'z_r2_3': 72,
    'x_s': 1.0,
    'x_s_1': 0.8999999999998373,
    'x_s_2': 0.7999999999988541,
    'x_s_3': -0.999999999999853,
    'x_r2': 0.9999999999985899,
    'x_r2_1': 0.719421838988003,
    'x_r2_2': 0.6194218389874829,
    'x_r2_3': 0.5194218389853328,
}

# Checking against:
paper_vals = [6, 81, .476, 1.0]

In [68]:
int_bounds = {
    "z_sh": [5, 15], #NOTE - z_sh is actually half z_s (which must be even)
    "z_r2": [70, 86],
}

float_bounds = {
    "x_s": [-0.5, 0.5],
    "x_r2": [-1.5, 1.5],
    "Cl" : [0.1, 0.4],
}

# manual overwrite to minimize run time.
sun_options = [6,7,8,9,10] # note, these are z_sh (half z_s)
r2_options = [70,71,72,73,74,75] # NOTE - add 71??
xs_options = [1,0.9,0.8,0.4,0.3,-1, -0.9, -0.7]
xr2_options = [1, 0.9, 0.8, 0.7, 0.6, 0.5]
Cl = [0.1e-3, 0.2e-3, 0.3e-3, 0.4e-3]

# Main

In [69]:
#generate all possible subsets
num_per_param = 4

# all_params = [sun_options, p2_options, r2_options, xs_options, xp1_options, xp2_options, xr2_options]
all_params = [sun_options, r2_options, xs_options, xr2_options, Cl]

ss_vals = []

for param in all_params:
    ss_vals.append(list(itertools.combinations(param, num_per_param)))

test_len = total_combinations = math.prod(len(subsets) for subsets in ss_vals)

# print(f'Total designs to test: {len(list(itertools.product(*ss_vals)))}')
print(f'Total designs to test: {test_len}')

Total designs to test: 78750


In [70]:
best_score = num_per_param**len(all_params)
best_score

1024

In [71]:
the_list = list(itertools.product(*ss_vals))

In [72]:
list(np.array(the_list[3]))

[array([6., 7., 8., 9.]),
 array([70., 71., 72., 73.]),
 array([1. , 0.9, 0.8, 0.4]),
 array([1. , 0.9, 0.7, 0.6]),
 array([0.0001, 0.0002, 0.0003, 0.0004])]

In [73]:
Max_goal = int(1e6) # most values we'd want
ss = int(np.round(test_len/Max_goal)) # required subsample
print(f'subsample: {ss}')

subsample: 0


In [74]:
# score the subsets, keep the highest score
high_score = 0
best_vals = []
match_vals = []

if ss > 1:
    # do a subset of all combinations:
    for i in range(Max_goal-1):
        score = score_vals(list(np.array(the_list[i*ss])), add_gear = False)
        if score > high_score:
            best_vals = the_list[i*ss]
            high_score = score

else:
    # do all combinations
    for combination_of_vals in itertools.product(*ss_vals):
        score = score_vals(list(np.array(combination_of_vals)), add_gear = False)
        if score > high_score:
            best_vals = list(np.array(combination_of_vals))
            high_score = score
        elif score == best_score:
            next_best = list(np.array(combination_of_vals))
            match_vals.append(next_best)


In [75]:
np.array(best_vals)

array([[ 6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00],
       [ 7.2e+01,  7.3e+01,  7.4e+01,  7.5e+01],
       [ 1.0e+00,  9.0e-01,  8.0e-01, -1.0e+00],
       [ 1.0e+00,  9.0e-01,  8.0e-01,  7.0e-01],
       [ 1.0e-04,  2.0e-04,  3.0e-04,  4.0e-04]])

In [76]:
match_vals

[]

In [77]:
high_score

np.float64(39254.85427373696)

In [78]:
# check scores:
b_high_score = score_vals(param_to_list(BayzOp_Highest, add_cl = True))
b_opt_score = score_vals(param_to_list(BayzOp_Optimal, add_cl = True))
print("BayzOp Highest Score: ", b_high_score)
print("BayzOp Optimal Score: ", b_opt_score)

BayzOp Highest Score:  15181.614117998011
BayzOp Optimal Score:  15197.781765273987


In [79]:
param_to_list(BayzOp_Highest, add_cl = True)

[array([15, 11, 10]),
 array([72, 71, 70]),
 array([1.      , 0.380826, 0.280826]),
 array([-0.423252, -0.523252, -1.      ]),
 array([0.0002, 0.0003, 0.0004])]